In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import pickle

In [2]:
# config of inputs
out_dir = 'data'
out_version = 'v0'

In [3]:
df_collection = pickle.load(open( "{}/Xy_collection_{}.pickle".format(out_dir, out_version), "rb" ))

In [4]:
df_collection.keys()

dict_keys(['X_train_raw_StandardScaler', 'X_test_pca_mini_MinMaxScaler', 'X_train_pca_mini', 'X_test_pca_full_MinMaxScaler', 'X_test_pca_mini_StandardScaler', 'X_train_pca_trunk_StandardScaler', 'X_test_pca_mini', 'X_valid_pca_trunk_StandardScaler', 'X_valid_raw', 'X_valid_pca_full_MinMaxScaler', 'X_valid_ica_mini_StandardScaler', 'X_test_pca_full', 'X_valid_ica_full_MinMaxScaler', 'X_train_ica_full', 'X_test_raw_StandardScaler', 'X_test_raw', 'X_test_ica_full_StandardScaler', 'y_train', 'X_train_pca_trunk_MinMaxScaler', 'X_train_raw', 'X_valid_pca_mini_MinMaxScaler', 'X_train_raw_MinMaxScaler', 'X_valid_pca_trunk_MinMaxScaler', 'X_test_pca_trunk', 'X_test_pca_trunk_StandardScaler', 'X_valid_pca_trunk', 'X_train_ica_mini_MinMaxScaler', 'X_train_pca_full_StandardScaler', 'X_valid_ica_full', 'X_train_ica_mini', 'X_test_ica_full', 'X_train_pca_trunk', 'X_train_pca_mini_StandardScaler', 'X_train_pca_full', 'X_train_pca_full_MinMaxScaler', 'X_valid_pca_full', 'X_test_pca_full_StandardScaler

In [5]:
transforms = [s[len('X_train_'):] for s in  df_collection.keys() if 'X_train' in s]

In [6]:
transforms

['raw_StandardScaler',
 'pca_mini',
 'pca_trunk_StandardScaler',
 'ica_full',
 'pca_trunk_MinMaxScaler',
 'raw',
 'raw_MinMaxScaler',
 'ica_mini_MinMaxScaler',
 'pca_full_StandardScaler',
 'ica_mini',
 'pca_trunk',
 'pca_mini_StandardScaler',
 'pca_full',
 'pca_full_MinMaxScaler',
 'ica_full_StandardScaler',
 'pca_mini_MinMaxScaler',
 'ica_mini_StandardScaler',
 'ica_full_MinMaxScaler']

# XGBoost

In [10]:
# Added to suppress warnings from model fiting of type
# DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. 
import warnings
warnings.filterwarnings('ignore', 'The truth value of an empty array is ambiguous. .*')

In [8]:
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

from sklearn.metrics import make_scorer,roc_auc_score

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [11]:
cv_results=[]
xgb_param_test_list = [{'max_depth': [2,3,4,5,6,8,10],
                    'min_child_weight': sp_randint(1, 10),
                    'subsample': sp_uniform(loc=0.6, scale=0.4), 
                    'colsample_bytree': sp_uniform(loc=0.85, scale=0.15),
                    'reg_alpha': [0, 1e-5, 1e-3, 1e-1, 1, 3, 5, 10],
                    'reg_lambda': [0, 1e-5, 1e-3, 1e-1, 1, 5, 10]}]

n_iter = [500]
clf=xgb.XGBClassifier(max_depth=-1, random_state=314, seed=314, silent=True, metric='None', n_jobs=2)

for n, param_test in zip(n_iter, xgb_param_test_list):
    print('============================ ', n, ' =================')
    gs1 = RandomizedSearchCV(estimator=clf, param_distributions=param_test, 
                             n_iter=n,
                             scoring=make_scorer(accuracy_score, greater_is_better=True),
                             cv=4,
                             random_state=314,
                             verbose=True)
    #gs1.fit(df_collection['X_train_{}'.format('raw')] , df_collection['y_train'])
    gs1.fit(pd.concat([df_collection['X_train_{}'.format('raw')],
                       df_collection['X_valid_{}'.format('raw')]]), 
            pd.concat([df_collection['y_train'],
                       df_collection['y_valid']]))
    print(gs1.best_params_)
    print(gs1.best_score_)
    cv_results.append(gs1.cv_results_)

============================  500  =================
Fitting 4 folds for each of 500 candidates, totalling 2000 fits
{'subsample': 0.7874833473239866, 'max_depth': 10, 'reg_lambda': 5, 'min_child_weight': 2, 'colsample_bytree': 0.9980714989053638, 'reg_alpha': 0.1}
0.843644544431946


[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:  1.5min finished


In [12]:
for i in np.argsort(cv_results[0]['mean_test_score'])[-5:]:
    print('{1} {2}   :  {0}'.format(cv_results[0]['params'][i], 
                                    cv_results[0]['mean_test_score'][i], 
                                    cv_results[0]['mean_train_score'][i]))

0.8391451068616423 0.8743905074489782   :  {'subsample': 0.8096846675195573, 'max_depth': 5, 'reg_lambda': 0, 'min_child_weight': 8, 'colsample_bytree': 0.9333099791181296, 'reg_alpha': 0.1}
0.8391451068616423 0.8871403262332798   :  {'subsample': 0.7557660954844897, 'max_depth': 5, 'reg_lambda': 1e-05, 'min_child_weight': 4, 'colsample_bytree': 0.8706589383900514, 'reg_alpha': 1}
0.8402699662542182 0.8758914461688074   :  {'subsample': 0.8837227061346042, 'max_depth': 10, 'reg_lambda': 0.1, 'min_child_weight': 8, 'colsample_bytree': 0.9336018589755691, 'reg_alpha': 0}
0.8413948256467941 0.8833910747329039   :  {'subsample': 0.7905535081311607, 'max_depth': 8, 'reg_lambda': 1e-05, 'min_child_weight': 6, 'colsample_bytree': 0.9356188122231713, 'reg_alpha': 1}
0.843644544431946 0.9066373344859101   :  {'subsample': 0.7874833473239866, 'max_depth': 10, 'reg_lambda': 5, 'min_child_weight': 2, 'colsample_bytree': 0.9980714989053638, 'reg_alpha': 0.1}


/home/mlisovyi/anaconda2/envs/Titanic/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [13]:
from sklearn.model_selection import cross_validate

In [14]:
score = cross_validate(xgb.XGBClassifier(max_depth=3, random_state=314, silent=True, metric='None', n_jobs=2),
                       X=df_collection['X_train_{}'.format('raw')],
                       y=df_collection['y_train'],
                       scoring=make_scorer(accuracy_score, greater_is_better=True),
                       cv=4,
                       return_train_score=True)

In [15]:
score

{'fit_time': array([0.03130794, 0.03370237, 0.02735114, 0.02837324]),
 'score_time': array([0.00253081, 0.00179887, 0.00180984, 0.00253487]),
 'test_score': array([0.80769231, 0.80128205, 0.82580645, 0.81818182]),
 'train_score': array([0.89247312, 0.89677419, 0.89270386, 0.90364026])}

# LightGBM

In [16]:
from lightgbm.sklearn import LGBMClassifier

In [31]:
lgb_cv_results = [] 
lgb_param_test_list = [{'num_leaves': sp_randint(6, 10),
                    'min_child_weight': sp_randint(1, 50),
                    'colsample_bytree': sp_uniform(loc=0.7, scale=0.3), 
                    'subsample': sp_uniform(loc=0.7, scale=0.3),
                    'reg_alpha': [0, 1e-5, 1e-3, 1e-1, 1, 2],
                    'reg_lambda': [0, 1e-5, 1e-3, 1e-1, 1, 2]}]

n_iter = [500]
clf=lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=2)

for n, param_test in zip(n_iter, lgb_param_test_list):
    print('============================ ', n, ' =================')
    gs2 = RandomizedSearchCV(estimator=clf, param_distributions=param_test, 
                             n_iter=n,
                             scoring=make_scorer(accuracy_score, greater_is_better=True),
                             cv=4,
                             random_state=314,
                             verbose=True)
    #gs2.fit(df_collection['X_train_{}'.format('raw')] , df_collection['y_train'])
    gs2.fit(pd.concat([df_collection['X_train_{}'.format('raw')],
                       df_collection['X_valid_{}'.format('raw')]]), 
            pd.concat([df_collection['y_train'],
                       df_collection['y_valid']]))
    print(gs2.best_params_)
    print(gs2.best_score_)
    lgb_cv_results.append(gs2.cv_results_)

============================  500  =================
Fitting 4 folds for each of 500 candidates, totalling 2000 fits
{'subsample': 0.9719454235733966, 'reg_alpha': 1, 'reg_lambda': 0, 'min_child_weight': 3, 'colsample_bytree': 0.874435683963595, 'num_leaves': 7}
0.843644544431946


[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:  1.1min finished


In [32]:
for i in np.argsort(lgb_cv_results[0]['mean_test_score'])[-5:]:
    print('{1} {2}   :  {0}'.format(lgb_cv_results[0]['params'][i], 
                                    lgb_cv_results[0]['mean_test_score'][i], 
                                    lgb_cv_results[0]['mean_train_score'][i]))

0.8402699662542182 0.8875145760453107   :  {'subsample': 0.7286759755581009, 'reg_alpha': 0.001, 'reg_lambda': 2, 'min_child_weight': 4, 'colsample_bytree': 0.825499562969686, 'num_leaves': 9}
0.8402699662542182 0.8867666392029211   :  {'subsample': 0.7635777990829171, 'reg_alpha': 0.001, 'reg_lambda': 0, 'min_child_weight': 4, 'colsample_bytree': 0.7651577542988559, 'num_leaves': 7}
0.8402699662542182 0.8882653267960615   :  {'subsample': 0.8370598352879277, 'reg_alpha': 1, 'reg_lambda': 0, 'min_child_weight': 2, 'colsample_bytree': 0.766300383073703, 'num_leaves': 8}
0.8413948256467941 0.8995108301705004   :  {'subsample': 0.8291372597673844, 'reg_alpha': 0, 'reg_lambda': 0.001, 'min_child_weight': 4, 'colsample_bytree': 0.9117493957358069, 'num_leaves': 9}
0.843644544431946 0.8878893886390138   :  {'subsample': 0.9719454235733966, 'reg_alpha': 1, 'reg_lambda': 0, 'min_child_weight': 3, 'colsample_bytree': 0.874435683963595, 'num_leaves': 7}


/home/mlisovyi/anaconda2/envs/Titanic/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
